In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.09.24 데이터 추출 완료 - 31140 데이터가 대부분 없음
cursor = mydb.cursor(prepared=True)
citycode = '32020'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20210924 | location : 31250 | cnt : 1 | start GGB204000025 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 2 | start GGB222000105 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 3 | start GGB234000011 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 4 | start GGB234000012 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 5 | start GGB234000024 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 6 | start GGB234000034 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 7 | start GGB234000039 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 8 | start GGB234000040 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 9 | start GGB234000057 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 10 | start GGB234000059 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 11 | start GGB234000061 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt 

조회날짜 : 20210924 | location : 31250 | cnt : 95 | start GGB234001486 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 96 | start GGB234001487 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 97 | start GGB234001497 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 98 | start GGB234001500 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 99 | start GGB234001504 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 100 | start GGB234001507 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 101 | start GGB234001509 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 102 | start GGB234001515 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 103 | start GGB234001516 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 104 | start GGB234001524 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 105 | start GGB234001540 public_busroute_info
조회날짜 : 20210924 | location

조회날짜 : 20210924 | location : 31250 | cnt : 188 | start GGB234001008 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 189 | start GGB234001015 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 190 | start GGB234001016 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 191 | start GGB234001021 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 192 | start GGB234001023 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 193 | start GGB234001028 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 194 | start GGB234001056 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 195 | start GGB234001104 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 196 | start GGB234001106 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 197 | start GGB234001107 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 198 | start GGB234001108 public_busroute_info
조회날짜 : 20210924 | loc

조회날짜 : 20210924 | location : 31250 | cnt : 281 | start GGB234001542 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 282 | start GGB234001543 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 283 | start GGB234001544 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 284 | start GGB234001545 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 285 | start GGB234001550 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 286 | start GGB234001575 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 287 | start GGB234001594 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 288 | start GGB234001596 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 289 | start GGB234001597 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 290 | start GGB234001618 public_busroute_info
조회날짜 : 20210924 | location : 31250 | cnt : 291 | start GGB234001642 public_busroute_info
조회날짜 : 20210924 | loc

조회날짜 : 20210924 | location : 31270 | cnt : 18 | start GGB236000218 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 19 | start GGB236000219 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 20 | start GGB236000224 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 21 | start GGB236000227 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 22 | start GGB236000228 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 23 | start GGB236000229 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 24 | start GGB236000231 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 25 | start GGB236000044 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 26 | start GGB236000048 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 27 | start GGB236000052 public_busroute_info
조회날짜 : 20210924 | location : 31270 | cnt : 28 | start GGB236000057 public_busroute_info
조회날짜 : 20210924 | location : 312

...

조회날짜 : 20210924 | location : 31380 | cnt : 55 | start GGB240000007 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 56 | start GGB240000009 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 57 | start GGB240000017 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 58 | start GGB240000029 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 59 | start GGB240000033 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 60 | start GGB240000037 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 61 | start GGB240000135 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 62 | start GGB240000139 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 63 | start GGB240000140 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 64 | start GGB240000144 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 65 | start GGB240000151 public_busroute_info
조회날짜 : 20210924 | location : 313

조회날짜 : 20210924 | location : 31380 | cnt : 150 | start GGB240000046 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 151 | start GGB240000086 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 152 | start GGB240000087 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 153 | start GGB240000092 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 154 | start GGB240000102 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 155 | start GGB240000093 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 156 | start GGB240000094 public_busroute_info
조회날짜 : 20210924 | location : 31380 | cnt : 157 | start GGB240000111 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 1 | start CCB250052501 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 2 | start CCB250052502 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 3 | start CCB250052503 public_busroute_info
조회날짜 : 20210924 | location 

조회날짜 : 20210924 | location : 32010 | cnt : 88 | start CCB250051702 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 89 | start CCB250051703 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 90 | start CCB250051800 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 91 | start CCB250051801 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 92 | start CCB250051900 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 93 | start CCB250052000 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 94 | start CCB250052001 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 95 | start CCB250052002 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 96 | start CCB250052003 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 97 | start CCB250052004 public_busroute_info
조회날짜 : 20210924 | location : 32010 | cnt : 98 | start CCB250052005 public_busroute_info
조회날짜 : 20210924 | location : 320

조회날짜 : 20210924 | location : 32010 | cnt : 182 | start CCB250000900 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 1 | start WJB251000004 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 2 | start WJB251000052 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 3 | start WJB251000069 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 4 | start WJB251000071 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 5 | start WJB251000073 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 6 | start WJB251000074 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 7 | start WJB251000076 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 8 | start WJB251000085 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 9 | start WJB251000089 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 10 | start WJB251000090 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt

조회날짜 : 20210924 | location : 32020 | cnt : 94 | start WJB251000198 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 95 | start WJB251000200 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 96 | start WJB251000215 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 97 | start WJB251000256 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 98 | start WJB251000257 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 99 | start WJB251000323 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 100 | start WJB251000324 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 101 | start WJB251000343 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 102 | start WJB251001007 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 103 | start WJB251001008 public_busroute_info
조회날짜 : 20210924 | location : 32020 | cnt : 104 | start WJB251001051 public_busroute_info
조회날짜 : 20210924 | location 